In [1]:
import numpy as np

In [ ]:
def discretize(Z, Zdim, Zmin, Zstep):
    ###
    #Discretize a variable Z using its dimension Zdim, its minimal values along each axis and the discretization steps
    res = [0]*Zdim #n-dimensional index
    for i in range(Zdim): #For each dimension
        elt = Z[i] #Extract the i-th element
        ind = int((elt - Zmin[i])//Zstep[i]) #Discretize
        res[i] = ind
    return(tuple(res)) #Return as tuple for array indexing

In [ ]:
class Pendulum():
    def __init__(self):
        self.l = 0.6
        self.m = 0.5
        self.max_torque = 10
        self.max_speed = 8
        self.state = np.array([0,0])
        self.last_input = None
        self.g = 9.81
        self.dt = 0.1
        self.angle_std = 3*2*np.pi/360
        self.speed_std = 0.1
        
    def random_init(self):
        self.state = np.array([np.random.uniform(-np.pi, np.pi),np.random.uniform(-sefl.max_speed, self.max_speed)])
        
    def step(self, u):
        u = np.clip(u, -self.max_torque, self.max_torque)
        self.last_input = u
        accel = u/(self.m*self.l*self.l) + self.g*np.sin(self.state[0])/self.l
        #print("Pr p: " + str(self.state[0]))
        speed = self.state[1]
        angle = self.state[0]
        
        new_speed = speed + accel*self.dt + np.random.normal(0,self.speed_std)
        new_speed = np.clip(new_speed, -self.max_speed, self.max_speed)
        
        new_angle = angle + new_speed*self.dt + np.random.normal(0,self.angle_std)
        if new_angle < -np.pi:
            new_angle = new_angle + 2*np.pi
        if new_angle > np.pi:
            new_angle = new_angle - 2*np.pi
        
        self.state = np.array([new_angle, new_speed])
        
    def set_state(self, angle, speed):
        speed = np.clip(speed, -self.max_speed, self.max_speed)
        if angle < -np.pi:
            angle = angle + 2*np.pi
        if angle > np.pi:
            angle = angle - 2*np.pi
        self.state = np.array([angle, speed])

        
P = Pendulum()

In [2]:
import numpy as np
import sys
from casadi import *
import do_mpc


model_type = 'continuous' # either 'discrete' or 'continuous'
model = do_mpc.model.Model(model_type)

ModuleNotFoundError: No module named 'casadi'

In [ ]:
theta = model.set_variable(var_type='_x', var_name='theta', shape=(1,1))
dtheta = model.set_variable(var_type='_x', var_name='dtheta', shape=(1,1))

u = model.set_variable(var_type='_u', var_name='u', shape=(1,1))

In [ ]:
m = P.m
l = P.l

g = P.g

In [ ]:
model.set_rhs('theta',dtheta)
model.set_rhs('dtheta', g*sin(theta)/l + u/(m*l*l))

In [ ]:
model.setup()

In [ ]:
mpc = do_mpc.controller.MPC(model)

In [ ]:
setup_mpc = {
    'n_horizon': 20,
    't_step': P.dt,
    'n_robust': 0,
    'store_full_solution': True,
}
mpc.set_param(**setup_mpc)

In [ ]:
l_term = theta*theta + 0.1*dtheta*dtheta
m_term = theta*theta + 0.5*dtheta*dtheta
mpc.set_objective(mterm=m_term, lterm=l_term)

In [ ]:
mpc.bounds['lower','_u', 'u'] = -2
mpc.bounds['upper','_u', 'u'] = 2

In [ ]:
mpc.setup()

In [ ]:
x0 = np.array([np.pi, 0]).reshape(-1,1)
mpc.x0 = x0

In [ ]:
mpc.set_initial_guess()

In [ ]:
#Checking that everything is fine by performing a test step
u0 = mpc.make_step(x0)

In [ ]:
mpc.reset_history()

In [ ]:
%%capture

nSteps = 300
hist = [[0,0]]*nSteps
uhist = [0]*nSteps

P = Pendulum()
P.set_state(np.pi,0)
for i in range(nSteps-1):
    state = P.state
    x0 = state.reshape(-1,1)
    u0 = mpc.make_step(x0)
    u = u0[0][0] + np.random.normal(0,0.2)
    P.step(u)
    hist[i+1] = state
    uhist[i] = u

In [ ]:
import matplotlib.pyplot as plt
#On the x axis: time step (each step being 0.1s, the experiment lasts 30s)
plt.plot(hist[1:])

In [ ]:
plt.plot([x[0] for x in hist[1:]])
#Separate plot of the angular position

In [ ]:
#The following cells perform 10 simulations and plot the average history with the shaded area representing the std

In [ ]:
nSims = 10
fullH = np.zeros((nSims,299))
uH = np.zeros((nSims,299))
nSteps = 300

for j in range(nSims):
    if j%10==0:
        print(j)
    hist = [[0,0]]*nSteps
    uhist = [0]*nSteps

    P = Pendulum()
    P.set_state(np.pi,0)
    mpc.reset_history()
    for i in range(nSteps-1):
        state = P.state
        x0 = state.reshape(-1,1)
        u0 = mpc.make_step(x0)
        u = u0[0][0] + np.random.normal(0,0.2)
        u = u - ((-2.5+u)%0.25)
        P.step(u)
        hist[i+1] = state
        uhist[i] = u
    fullH[j] = [x[0] for x in hist[1:]]
    uH[j] = uhist[:299]

In [ ]:
means = [0]*299
stds = [0]*299
for i in range(299):
    means[i] = np.mean(fullH[:,i])
    stds[i] = np.std(fullH[:,i])

In [ ]:
from matplotlib import pyplot as plt
from pylab import rcParams
plt.rcParams.update({'font.size': 18})

x = np.array([x/10 for x in range(299)])
y = np.array(means)
ci = np.array(stds)

fig, ax = plt.subplots()
ax.plot(x,y)
ax.fill_between(x, (y-ci), (y+ci), color='b', alpha=.1)
#ax.set_title('FPD input')
ax.set_xlabel('Time')
ax.set_ylabel('Angular position')
plt.savefig('mpc_angle_ci.png', bbox_inches = 'tight')

In [ ]:
umeans = [0]*299
ustds = [0]*299
for i in range(299):
    umeans[i] = np.mean(uH[:,i])
    ustds[i] = np.std(uH[:,i])

In [ ]:
from matplotlib import pyplot as plt

x = np.array([x/10 for x in range(299)])
y = np.array(umeans)
ci = np.array(ustds)

fig, ax = plt.subplots()
ax.plot(x,y)
ax.fill_between(x, (y-ci), (y+ci), color='b', alpha=.1)
#ax.set_title('FPD input')
ax.set_xlabel('Time')
ax.set_ylabel('Control input')
plt.savefig('mpc_input_ci.png', bbox_inches = 'tight')